## 2023 Open Science Data Challenge - Sentinel-2 Phenology with Cloud Filtering

This notebook calculates vegetation phenology using Sentinel-2 data with cloud filtering. This pixel-based approach is better than a scene-based approach since clouds are quite random for any given time and location. To address phenology, the algorithm uses the Normalized Difference Vegetation Index (NDVI) which is a common proxy for vegetation growth and health. The outputs of this notebook can be used to assess differences in agriculture fields over time or space and also allow the assessment of growing states such as planting and harvesting. The baseline data is [Sentinel-2 Level-2A](https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a) data from the MS Planetary Computer catalog.

In [1]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rasterio.features
import rioxarray as rio
import xrspatial.multispectral as ms

# Import Planetary Computer tools
import pystac_client
import planetary_computer as pc
import odc
from odc.stac import stac_load
from odc.algo import to_rgba

from tqdm import tqdm

In [2]:
crop_presence_data = pd.read_csv("Crop_Location_Data_20221201.csv")
crop_presence_data

,Latitude and Longitude,Class of Land
0,"(10.323727047081501, 105.2516346045924)",Rice
1,"(10.322364360592521, 105.27843410554115)",Rice
2,"(10.321455902933202, 105.25254306225168)",Rice
3,"(10.324181275911162, 105.25118037576274)",Rice
4,"(10.324635504740822, 105.27389181724476)",Rice
...,...,...
595,"(10.013942985253381, 105.67361318732796)",Non Rice
596,"(10.01348875642372, 105.67361318732796)",Non Rice
597,"(10.013034527594062, 105.67361318732796)",Non Rice
598,"(10.012580298764401, 105.67361318732796)",Non Rice


In [3]:
crop_presence_data['Latitude and Longitude']=crop_presence_data['Latitude and Longitude'].apply(lambda x: x.replace('(','').replace(')','').replace(' ','').split(','))

In [14]:
for i in range(len(crop_presence_data)):
    crop_presence_data['Latitude'][i]=crop_presence_data['Latitude and Longitude'][i][0]
    crop_presence_data['Longitude'][i]=crop_presence_data['Latitude and Longitude'][i][1]

In [15]:
crop_presence_data

,Latitude and Longitude,Class of Land,Latitude,Longitude
0,"[10.323727047081501, 105.2516346045924]",Rice,10.323727047081501,105.2516346045924
1,"[10.322364360592521, 105.27843410554115]",Rice,10.322364360592521,105.27843410554115
2,"[10.321455902933202, 105.25254306225168]",Rice,10.321455902933202,105.25254306225168
3,"[10.324181275911162, 105.25118037576274]",Rice,10.324181275911162,105.25118037576274
4,"[10.324635504740822, 105.27389181724476]",Rice,10.324635504740822,105.27389181724476
...,...,...,...,...
595,"[10.013942985253381, 105.67361318732796]",Non Rice,10.013942985253381,105.67361318732796
596,"[10.01348875642372, 105.67361318732796]",Non Rice,10.01348875642372,105.67361318732796
597,"[10.013034527594062, 105.67361318732796]",Non Rice,10.013034527594062,105.67361318732796
598,"[10.012580298764401, 105.67361318732796]",Non Rice,10.012580298764401,105.67361318732796


In [17]:
crop_presence_data= crop_presence_data.drop(columns=['Latitude and Longitude'])

In [18]:
crop_presence_data.columns

Index(['Class of Land', 'Latitude', 'Longitude'], dtype='object')

In [19]:
crop_presence_data

,Class of Land,Latitude,Longitude
0,Rice,10.323727047081501,105.2516346045924
1,Rice,10.322364360592521,105.27843410554115
2,Rice,10.321455902933202,105.25254306225168
3,Rice,10.324181275911162,105.25118037576274
4,Rice,10.324635504740822,105.27389181724476
...,...,...,...
595,Non Rice,10.013942985253381,105.67361318732796
596,Non Rice,10.01348875642372,105.67361318732796
597,Non Rice,10.013034527594062,105.67361318732796
598,Non Rice,10.012580298764401,105.67361318732796


In [ ]:
crop_presence_data.to_excel('class of Land_.xlsx', index=False)

In [ ]:
NDVI_values=[]
for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
    lat_long=coordinates.replace('(','').replace(')','').replace(' ','').split(',')
    #box_size_deg = 0
    min_lon = lat_long[1]
    min_lat = lat_long[0]
    max_lon = lat_long[1]
    max_lat = lat_long[0]
    bounds = (min_lon, min_lat, max_lon, max_lat)
    time_window="2021-12-01/2022-01-31"
    stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = stac.search(collections=["sentinel-2-l2a"], bbox=bounds, datetime=time_window)
    items = list(search.get_all_items())
    resolution = 20  # meters per pixel 
    scale = resolution / 111320.0 # degrees per pixel for CRS:4326 
    xx = stac_load(
    items,
    bands=["red", "nir", "SCL"],
    crs="EPSG:4326", # Latitude-Longitude
    resolution=scale, # Degrees
    chunks={"x": 2048, "y": 2048},
    dtype="uint16",
    patch_url=pc.sign,
    bbox=bounds)
    cloud_mask = \
    (xx.SCL != 0) & \
    (xx.SCL != 1) & \
    (xx.SCL != 3) & \
    (xx.SCL != 6) & \
    (xx.SCL != 8) & \
    (xx.SCL != 9) & \
    (xx.SCL != 10) 
    cleaned_data = xx.where(cloud_mask).astype("uint16")
    mean_unfiltered = xx.mean(dim=['longitude','latitude']).compute()
    ndvi_mean = (mean_unfiltered.nir-mean_unfiltered.red)/(mean_unfiltered.nir+mean_unfiltered.red)
    mean_clean = cleaned_data.mean(dim=['longitude','latitude']).compute()
    ndvi_mean_clean = (mean_clean.nir-mean_clean.red)/(mean_clean.nir+mean_clean.red)
    nparray_ndvi_mean_clean=np.array(ndvi_mean_clean.values)
    nan_mask = np.isnan(nparray_ndvi_mean_clean)
    non_nan_nparray_ndvi_mean_clean = nparray_ndvi_mean_clean[~nan_mask]
    NDVI_values.append(non_nan_nparray_ndvi_mean_clean)
    print(lat_long)

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

In [ ]:
NDVI_values